---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [ ]:
def date_sorter():
#     df = pd.DataFrame(df, columns=['text'])
#     c = df[:].str.findall(r'(?:\d{1,2}(?:-|/|. |, | ){1})?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|\d{1,2})[a-z]*(?:-|/|. |, | )?(?:\d{1,2}[a-z]*(?:-|/|. |, | )?)?\d{2,4}')
#     c = df[:].str.findall(r'(?:\d{1,2} )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|\d{1,2})[a-z]*(?:-|/| |. )(?:\d{1,2}[a-z]{0,2}(?:-|/|, | ))?\d{2,4}')
#     c = pd.DataFrame(c)
#     c.rename(columns={0: "date"},inplace=True)
#     c['date']=c['date'].apply(lambda x: (x[0]))
#     c = c.apply(pd.to_datetime)
    import numpy as np
    df_new = pd.DataFrame(df)
    c = df[:].str.extractall(r'(\d{1,2}\s)?(?:(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|\d{1,2})[a-z]*(?:-|/|,\s|.\s|\s))?(?:(\d{1,2})[a-z]{0,2}(?:-|/|,\s|\s))?(\d{2,4})')
    c.fillna(0, inplace=True)
    
    df_new["Date"] = ""
    for i in np.arange(len(df)):
#         print(i)
        if len(c.loc[i]) == 2:
            l = [1,0]
        else:
            l=[0]
        for j in l:
            
            if (len(c.loc[i,j][3]) == 2) & (int(c.loc[i,j][3]) > 70):
                year = '19'+(c.loc[i,j][3])
            elif (len(c.loc[i,j][3]) == 4) & (int(c.loc[i,j][3]) < 2050):
                year = (c.loc[i,j][3])
            else:
                year = '2020'
            if (len(str(c.loc[i,j][1])) < 3):
                if (int(c.loc[i,j][1]) < 13) & (int(c.loc[i,j][1]) > 0):
                    month = str(c.loc[i,j][1])
                elif (int(c.loc[i,j][1]) == 0):
                    month = '1'
            elif (len(str(c.loc[i,j][1])) == 3) & ((c.loc[i,j][1]) in ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']):
                month = str(c.loc[i,j][1])
            else:
                month = '12'
            if (len(str(c.loc[i,j][0])) < 3) & (int(c.loc[i,j][0]) < 30) & (int(c.loc[i,j][0]) > 0):
                day = (c.loc[i,j][0])
            elif (len(str(c.loc[i,j][2])) < 3) & (int(c.loc[i,j][2]) < 30) & (int(c.loc[i,j][2]) > 0):
                day = (c.loc[i,j][2])
            elif (int(c.loc[i,j][2]) == 0) | (int(c.loc[i,j][0]) == 0):
                day = '1'
            else:
                day='30'
            df_new.loc[i,'Date'] = month+'/'+day+'/'+year
    df_new["DateS"] = pd.to_datetime(df_new['Date'])
    for i in np.arange(len(df)):
        if df_new.loc[i,"DateS"].year == 2020:
            j = 1
            if (len(c.loc[i,j][3]) == 2) & (int(c.loc[i,j][3]) > 70):
                year = '19'+(c.loc[i,j][3])
            elif (len(c.loc[i,j][3]) == 4) & (int(c.loc[i,j][3]) < 2050):
                year = (c.loc[i,j][3])
            else:
                year = '2020'
            if (len(str(c.loc[i,j][1])) < 3):
                if (int(c.loc[i,j][1]) < 13) & (int(c.loc[i,j][1]) > 0):
                    month = str(c.loc[i,j][1])
                elif (int(c.loc[i,j][1]) == 0):
                    month = '1'
            elif (len(str(c.loc[i,j][1])) == 3) & ((c.loc[i,j][1]) in ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']):
                month = str(c.loc[i,j][1])
            else:
                month = '12'
            if (len(str(c.loc[i,j][0])) < 3) & (int(c.loc[i,j][0]) < 30) & (int(c.loc[i,j][0]) > 0):
                day = (c.loc[i,j][0])
            elif (len(str(c.loc[i,j][2])) < 3) & (int(c.loc[i,j][2]) < 30) & (int(c.loc[i,j][2]) > 0):
                day = (c.loc[i,j][2])
            elif (int(c.loc[i,j][2]) == 0) | (int(c.loc[i,j][0]) == 0):
                day = '1'
            else:
                day='30'
            df_new.loc[i,'Date'] = month+'/'+day+'/'+year
    df_new["DateS"] = pd.to_datetime(df_new['Date'])
    df_new.sort_values(by=["DateS"], inplace=True)
    return pd.Series(df_new.index)
date_sorter()

In [3]:
# def date_sorter_try():
#     a = ['04/20/2009', '04/20/09', '4/20/09', '4/3/09', 'Mar-20-2009', 'Mar 20, 2009', 'March 20, 2009','Mar. 20, 2009', 
#          'Mar 20 2009', '20 Mar 2009', '20 March 2009', '20 Mar. 2009', '20 March, 2009','Mar 20th, 2009', 'Mar 21st, 2009',
#           'Mar 22nd, 2009', 'Feb 2009', 'Sep 2009', 'Oct 2010', '6/2008', '12/2009','2009', '2010','90801','12 969 2007']
#     # Your code here
#     a = pd.DataFrame(a)
# #     b = a[0].str.findall(r'(?:\d{,2} )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* (?:\d{,2}, )?\d{2,4}')
# #     b = a[0].str.findall(r'(?:\d{,2}|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*? (?:\d{,2}, )?\d{2,4}')
# #     c = a[0].str.findall(r'(?:\d{,2}|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ .,/-]{0,1}(?:\d{,2})[ .,/-]{0,1}(?:\d{2,4})')
# #     c = a[0].str.findall(r'(?:\d{0,2}|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[ .,/-]{0,2}(?:\d{0,2})[ ./-|,]{0,2}?(?:\d{2,4})')
# #     c = a[0].str.findall(r'(?:\d{1,2}(?:-|/|. |, | )?)?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|\d{1,2})[a-z]*(?:-|/|. |, | ){1}(?:\d{1,2}[a-z]*(?:-|/|. |, | )?)?\d{2,4}')
# #     c = a[0].str.findall(r'(?:\d{1,2}\s)?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|\d{1,2})[a-z]*(?:-|/|,\s|.\s|\s)?(?:\d{1,2}[a-z]{0,2}(?:-|/|,\s|\s))?(?:\d{2,4})')
#     c = a[0].str.findall(r'(?:\d{1,2}\s)?(?:(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|\d{1,2})[a-z]*(?:-|/|,\s|.\s|\s))?(?:(?:\d{1,2})[a-z]{0,2}(?:-|/|,\s|\s))?(?:\d{2,4})')
# #     c = a[0].str.findall(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|\d{1,2})[a-z]*(?:/|, | |. )')
# #     c = a[0].str.findall(r'(?:\d{1,2} )?(?:(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[^0-9]*|\d{1,2})(?:-|/| |. )(?:\d{2,4})')
#     #     c = a[0].str.findall(r'[.,/-]?[ ]?
#     c = pd.DataFrame(c)
#     c.rename(columns={0: "date"},inplace=True)
# #     c['date']=c['date'].apply(lambda x: (x[0]))
# #     d=pd.DataFrame(c['date'].apply(lambda x: (x[0])))
# #     print(type(d))
# #     c = c.apply(pd.to_datetime)
#     c['original'] = a[0]
#     return c # Your answer here
# c=(date_sorter_try())
# print(c)

In [4]:
c.tail(15)

,0,Date,DateS
413,11/2016 Primary Care Doctor:\n,11/1/2016,2016-11-01
320,50 yo man with history consistant with ADHD un...,1/1/2020,2020-01-01
87,)Dilantin (PHENYTOIN) 100 MG CAPSULE Take 2 Ca...,1/1/2020,2020-01-01
331,Prozac- highest dose 40 mg; currently on 20 mg...,1/1/2020,2020-01-01
339,"Currently on Zoloft, 50 mg, Dicolenac, 100 mg,...",1/1/2020,2020-01-01
214,s Gale Youngquist is a 22 yo single Caucasian ...,1/1/2020,2020-01-01
303,"7 first psych hospital approx age 20, most rec...",1/1/2020,2020-01-01
107,"Pt is a 21 year old, single, heterosexual iden...",1/1/2020,2020-01-01
297,"B/R Natchez, GA on a commune. Completed 1 year...",1/1/2020,2020-01-01
312,".On review of systems, she reports that her sl...",1/1/2020,2020-01-01


'. Psychosocial: lives w/ father, looking to get own apartment. Lives in Saluda. Fa. has roommate. Parents divorced when pt 5 y/o old. Mother lives in saluda, see her often, drives hm to work. Relationship okay w/ dad and mom is okay too. Has one sibling, age 18 y/o. Get on well currently. Close family on mother\'s side. No family issues. Employment: Woriking 63 hours - "I love it." At construction worker. School: finished HS - dipolma, started working after HS. Likes job, makes good money. Peers: have friends, no issues, all friends are sober from opiates but drink/smoke "they distance from me when I use." Single currently. Stressors: biggest stress - Mar, 1975 - uncle passed from heroin OD. Pt was close do him, was mother\'s brother. Finances - getting better, owe lots of money to people/banks too.\n'